In [13]:
import pandas as pd
import plotly_express as px
import plotly.graph_objects as go
import hashlib as hl
pd.set_option("display.max_rows", None)

In [14]:
directory_data = "../data"

athletes = pd.read_csv(f"{directory_data}/athlete_events.csv")
regions = pd.read_csv(f"{directory_data}/noc_regions.csv")

In [15]:
athletes["Name"] = athletes["Name"].astype(str)
hashes = athletes["Name"].apply(lambda x: hl.sha256(x.encode()).hexdigest())
hashes.head()

0    3a4eef48434c66b3f14ab0221f6762d0ef7c6135ab2790...
1    a6430cc6630934275dc6283f7e97e9625e6587cdddec7a...
2    9c198b205332c2c8e1542e0f9534b9e270780a41d978ec...
3    0a477bb1c5ad39716f9c775e54d18d16aa8b37ada55548...
4    5b7be356aa28178096dc6747f0b8e4e393eaceb5f95310...
Name: Name, dtype: object

In [16]:
athletes.insert(1, "Hash values", hashes)
athletes.head()

,ID,Hash values,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,3a4eef48434c66b3f14ab0221f6762d0ef7c6135ab2790...,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,a6430cc6630934275dc6283f7e97e9625e6587cdddec7a...,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,9c198b205332c2c8e1542e0f9534b9e270780a41d978ec...,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,0a477bb1c5ad39716f9c775e54d18d16aa8b37ada55548...,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,5b7be356aa28178096dc6747f0b8e4e393eaceb5f95310...,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [17]:
athletes.drop(columns=["Name"], inplace=True)
athletes.head()

,ID,Hash values,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,3a4eef48434c66b3f14ab0221f6762d0ef7c6135ab2790...,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,a6430cc6630934275dc6283f7e97e9625e6587cdddec7a...,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,9c198b205332c2c8e1542e0f9534b9e270780a41d978ec...,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,0a477bb1c5ad39716f9c775e54d18d16aa8b37ada55548...,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,5b7be356aa28178096dc6747f0b8e4e393eaceb5f95310...,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


### Fix age missing

In [18]:
# Load file seperate fix, but since it was from excel convertion was different so needed to drop ; 
fixAge = pd.read_csv(f"{directory_data}/australia_missing_AGE_FIX.csv", sep=";")

# Drop first column if it's unnamed/empty
if fixAge.columns[0].startswith("Unnamed"):
    fixAge = fixAge.drop(columns=[fixAge.columns[0]])

# Merge only the corrected ages
merged_age = athletes.merge(
    fixAge[['ID', 'Age']],  # include ID or unique key + Age
    on='ID',
    how='left',
    suffixes=('', '_fixed')
)

# Replace missing Age values with fixed ones
merged_age['Age'] = merged_age['Age'].fillna(merged_age['Age_fixed'])

# Drop helper column
merged_age = merged_age.drop(columns=['Age_fixed'])

# Save result
#merged_age.to_csv(f"{directory_data}/athlete_events_fixed_age.csv", index=False)

In [20]:
australiaAge = merged_age[merged_age["NOC"] == "AUS"]

nullsAge = australiaAge.isnull().sum()
print(nullsAge)

ID                0
Hash values       0
Sex               0
Age               0
Height         1113
Weight         1229
Team              0
NOC               0
Games             0
Year              0
Season            0
City              0
Sport             0
Event             0
Medal          6544
dtype: int64


In [21]:
australiaAge = merged_age[merged_age["NOC"] == "AUS"]

nullsAge = australiaAge.isnull().sum()
print(nullsAge)

ID                0
Hash values       0
Sex               0
Age               0
Height         1113
Weight         1229
Team              0
NOC               0
Games             0
Year              0
Season            0
City              0
Sport             0
Event             0
Medal          6544
dtype: int64


In [19]:
merged = pd.merge(athletes, regions, on="NOC", how="outer")
df = merged

In [11]:
df.describe()

AttributeError: 'str' object has no attribute 'describe'

In [20]:
df = df[df['Year'] != 1906]
df = df[df['Sport'] != "Alpinism"]
df.loc[(df["ID"] == 118133) & (df["Year"] == 2012), "Medal"] = "Gold"


In [26]:
#print(df[df["ID"].str.contains(118133, case=False, na=False)][["ID", "Year", "Event", "Medal"]])
print(df["ID"].dtype)

float64


In [24]:
"""
medals = (
    merged
    .dropna(subset=["Medal"])
    .groupby(["Year", "Medal"], as_index=False)
    .agg(MedalCount=("Medal", "count"))
)"""


'\nmedals = (\n    merged\n    .dropna(subset=["Medal"])\n    .groupby(["Year", "Medal"], as_index=False)\n    .agg(MedalCount=("Medal", "count"))\n)'

In [25]:
medals = df[df['Medal'].notna()]
print(medals['NOC'].unique())

['AFG' 'AHO' 'ALG' 'ANZ' 'ARG' 'ARM' 'AUS' 'AUT' 'AZE' 'BAH' 'BAR' 'BDI'
 'BEL' 'BER' 'BLR' 'BOH' 'BOT' 'BRA' 'BRN' 'BUL' 'CAN' 'CHI' 'CHN' 'CIV'
 'CMR' 'COL' 'CRC' 'CRO' 'CUB' 'CYP' 'CZE' 'DEN' 'DJI' 'DOM' 'ECU' 'EGY'
 'ERI' 'ESP' 'EST' 'ETH' 'EUN' 'FIJ' 'FIN' 'FRA' 'FRG' 'GAB' 'GBR' 'GDR'
 'GEO' 'GER' 'GHA' 'GRE' 'GRN' 'GUA' 'GUY' 'HAI' 'HKG' 'HUN' 'INA' 'IND'
 'IOA' 'IRI' 'IRL' 'IRQ' 'ISL' 'ISR' 'ISV' 'ITA' 'JAM' 'JOR' 'JPN' 'KAZ'
 'KEN' 'KGZ' 'KOR' 'KOS' 'KSA' 'KUW' 'LAT' 'LIB' 'LIE' 'LTU' 'LUX' 'MAR'
 'MAS' 'MDA' 'MEX' 'MGL' 'MKD' 'MNE' 'MON' 'MOZ' 'MRI' 'NAM' 'NED' 'NGR'
 'NIG' 'NOR' 'NZL' 'PAK' 'PAN' 'PAR' 'PER' 'PHI' 'POL' 'POR' 'PRK' 'PUR'
 'QAT' 'ROU' 'RSA' 'RUS' 'SCG' 'SEN' 'SGP' 'SLO' 'SRB' 'SRI' 'SUD' 'SUI'
 'SUR' 'SVK' 'SWE' 'SYR' 'TAN' 'TCH' 'TGA' 'THA' 'TJK' 'TOG' 'TPE' 'TTO'
 'TUN' 'TUR' 'UAE' 'UAR' 'UGA' 'UKR' 'URS' 'URU' 'USA' 'UZB' 'VEN' 'VIE'
 'WIF' 'YUG' 'ZAM' 'ZIM']


In [26]:
medals.head()

,ID,Hash values,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
64,86313.0,539474f285f370ca64bdc5fe642c308496a152aef22945...,M,21.0,183.0,63.0,Afghanistan,AFG,2008 Summer,2008.0,Summer,Beijing,Taekwondo,Taekwondo Men's Flyweight,Bronze,Afghanistan,NaN
65,86313.0,539474f285f370ca64bdc5fe642c308496a152aef22945...,M,25.0,183.0,63.0,Afghanistan,AFG,2012 Summer,2012.0,Summer,London,Taekwondo,Taekwondo Men's Featherweight,Bronze,Afghanistan,NaN
130,12684.0,f2d0dfea80d1e65fe750ea2f6d6961007d1112f015a636...,M,19.0,NaN,NaN,Netherlands Antilles,AHO,1988 Summer,1988.0,Summer,Seoul,Sailing,Sailing Mixed Windsurfer,Silver,Curacao,Netherlands Antilles
311,2794.0,054c3e790548e150f640abdfec3d655bca7d604d93bb12...,M,26.0,174.0,63.0,Algeria,ALG,2000 Summer,2000.0,Summer,Sydney,Boxing,Boxing Men's Light-Welterweight,Bronze,Algeria,NaN
333,6815.0,d0ef3cf00e3436a6c38973a5bd9726a3ad1471564bb7a2...,M,20.0,180.0,75.0,Algeria,ALG,1996 Summer,1996.0,Summer,Atlanta,Boxing,Boxing Men's Middleweight,Bronze,Algeria,NaN


In [ ]:
medals_filtered = medals.drop_duplicates(["Year", "Event", "Season", "Medal", "NOC"])
medals_filtered

In [29]:
aus_medals = medals_filtered.groupby(["region", "Year", "Season", "Games", "City", "Sport", "Sex", "Age", "Event"])["Medal"].value_counts().loc["Australia"].reset_index(name="MedalCount")
aus_medals

,Year,Season,Games,City,Sport,Sex,Age,Event,Medal,MedalCount
0,1896.0,Summer,1896 Summer,Athina,Athletics,M,22.0,"Athletics Men's 1,500 metres",Gold,1
1,1896.0,Summer,1896 Summer,Athina,Athletics,M,22.0,Athletics Men's 800 metres,Gold,1
2,1896.0,Summer,1896 Summer,Athina,Tennis,M,22.0,Tennis Men's Doubles,Bronze,1
3,1900.0,Summer,1900 Summer,Paris,Athletics,M,23.0,Athletics Men's 100 metres,Bronze,1
4,1900.0,Summer,1900 Summer,Paris,Athletics,M,23.0,Athletics Men's 200 metres,Bronze,1
5,1900.0,Summer,1900 Summer,Paris,Athletics,M,23.0,"Athletics Men's 5,000 metres, Team",Gold,1
6,1900.0,Summer,1900 Summer,Paris,Athletics,M,23.0,Athletics Men's 60 metres,Bronze,1
7,1900.0,Summer,1900 Summer,Paris,Swimming,M,20.0,Swimming Men's 200 metres Freestyle,Gold,1
8,1900.0,Summer,1900 Summer,Paris,Swimming,M,20.0,Swimming Men's 200 metres Obstacle Course,Gold,1
9,1904.0,Summer,1904 Summer,St. Louis,Swimming,M,22.0,Swimming Men's 220 yard Freestyle,Silver,1


In [ ]:
bronze_medal = medals_filtered.groupby(["Medal", "Year"])["NOC"].value_counts().loc[["Bronze"]]
bronze_aus = bronze_medal.xs("AUS", level="NOC")
bronze_aus

Medal   Year  
Bronze  1896.0     1
        1900.0     3
        1904.0     1
        1920.0     1
        1924.0     2
        1928.0     1
        1932.0     1
        1936.0     1
        1948.0     5
        1952.0     3
        1956.0    14
        1960.0     6
        1964.0    10
        1968.0     5
        1972.0     2
        1976.0     4
        1980.0     5
        1984.0    12
        1988.0     5
        1992.0    11
        1994.0     1
        1996.0    23
        1998.0     1
        2000.0    17
        2004.0    17
        2006.0     1
        2008.0    17
        2012.0    12
        2014.0     1
        2016.0    10
Name: count, dtype: int64

In [ ]:
bronze_anz = bronze_medal.xs("ANZ", level="NOC")
bronze_anz

Medal   Year  
Bronze  1908.0    2
        1912.0    3
Name: count, dtype: int64

In [ ]:
#medals_filtered.groupby(["Sport","NOC", "region", "Year"])["Medal"].value_counts().loc[["Alpinism"]]

In [ ]:
aus_2012 = medals_filtered[(medals_filtered["Year"] == 2012) & (medals_filtered["NOC"] == "AUS")]
aus_2012_silver = aus_2012[(aus_2012["Season"] == "Summer") & (aus_2012["Medal"] == "Silver")]
aus_2012_silver

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
5319,6934.0,Angie Lee Bainbridge,F,22.0,181.0,67.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Swimming,Swimming Women's 4 x 200 metres Freestyle Relay,Silver,Australia,NaN
5661,12490.0,Jack Bobridge,M,23.0,180.0,65.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Cycling,"Cycling Men's Team Pursuit, 4,000 metres",Silver,Australia,NaN
5893,15216.0,Brittany Broben,F,16.0,163.0,52.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Diving,Diving Women's Platform,Silver,Australia,NaN
6285,19847.0,James Jonathan Chapman,M,32.0,190.0,92.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Rowing,Rowing Men's Coxless Fours,Silver,Australia,NaN
6564,23519.0,Alicia Jayne Coutts,F,24.0,176.0,69.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Swimming,Swimming Women's 200 metres Individual Medley,Silver,Australia,NaN
6565,23519.0,Alicia Jayne Coutts,F,24.0,176.0,69.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Swimming,Swimming Women's 4 x 100 metres Medley Relay,Silver,Australia,NaN
6655,23988.0,"Kimberley ""Kim"" Crow-Brennan",F,26.0,188.0,74.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Rowing,Rowing Women's Double Sculls,Silver,Australia,NaN
6714,24422.0,Nina Curtis,F,24.0,167.0,68.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Sailing,Sailing Women's Three Person Keelboat,Silver,Australia,NaN
7431,36567.0,"Jessica Esther ""Jess"" Fox",F,18.0,166.0,60.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Canoeing,"Canoeing Women's Kayak Singles, Slalom",Silver,Australia,NaN
8314,49999.0,Kate Hornsey,F,30.0,174.0,73.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Rowing,Rowing Women's Coxless Pairs,Silver,Australia,NaN


In [ ]:
medals_filtered.at[11827, 'Medal'] = "Gold"
merged.at[11827, 'Medal'] = "Gold"

In [ ]:
aus_2012 = medals_filtered[(medals_filtered["Year"] == 2012) & (medals_filtered["NOC"] == "AUS")]
aus_2012_Gold = aus_2012[(aus_2012["Season"] == "Summer") & (aus_2012["Medal"] == "Gold")]
aus_2012_Gold

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
5512,9567.0,Mathew Belcher,M,29.0,173.0,62.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Sailing,Sailing Men's Two Person Dinghy,Gold,Australia,NaN
6152,17687.0,Cate Natalie Campbell,F,20.0,186.0,67.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Swimming,Swimming Women's 4 x 100 metres Freestyle Relay,Gold,Australia,NaN
6404,21966.0,Jacob Clear,M,27.0,185.0,86.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Canoeing,"Canoeing Men's Kayak Fours, 1,000 metres",Gold,Australia,NaN
8540,54381.0,Iain Jensen,M,24.0,186.0,80.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Sailing,Sailing Men's Skiff,Gold,Australia,NaN
9595,77809.0,Sally McLellan-Pearson,F,25.0,167.0,60.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Athletics,Athletics Women's 100 metres Hurdles,Gold,Australia,NaN
9651,78005.0,Anna Maree Devenish Meares,F,28.0,165.0,70.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Cycling,Cycling Women's Sprint,Gold,Australia,NaN
11438,111956.0,Tom Slingsby,M,27.0,186.0,81.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Sailing,Sailing Men's One Person Dinghy,Gold,Australia,NaN
11827,118133.0,Jared Tallent,M,27.0,178.0,58.0,Australia,AUS,2012 Summer,2012.0,Summer,London,Athletics,Athletics Men's 50 kilometres Walk,Gold,Australia,NaN


In [ ]:
medals["Medal"] = pd.Categorical(
    medals["Medal"],
    categories=["Gold", "Silver", "Bronze"],
    ordered=True
)

C:\Users\Pontus\AppData\Local\Temp\ipykernel_2148\3469094849.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medals["Medal"] = pd.Categorical(


In [ ]:
medals_total = medals.sort_values(["Year", "Medal"])

In [ ]:
medals_total.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
7350,35698.0,"Edwin Harold ""Teddy"" Flack",M,22.0,NaN,NaN,Australia,AUS,1896 Summer,1896.0,Summer,Athina,Athletics,Athletics Men's 800 metres,Gold,Australia,NaN
7351,35698.0,"Edwin Harold ""Teddy"" Flack",M,22.0,NaN,NaN,Australia,AUS,1896 Summer,1896.0,Summer,Athina,Athletics,"Athletics Men's 1,500 metres",Gold,Australia,NaN
15557,85477.0,Paul Neumann (-Newman),M,20.0,NaN,NaN,Austria,AUT,1896 Summer,1896.0,Summer,Athina,Swimming,Swimming Men's 500 metres Freestyle,Gold,Austria,NaN
16522,107006.0,Felix Adolf Schmal,M,23.0,NaN,NaN,Austria,AUT,1896 Summer,1896.0,Summer,Athina,Cycling,Cycling Men's 12-Hours Race,Gold,Austria,NaN
58007,54456.0,Alexander Viggo Jensen,M,21.0,NaN,NaN,Denmark,DEN,1896 Summer,1896.0,Summer,Athina,Weightlifting,"Weightlifting Men's Unlimited, Two Hands",Gold,Denmark,NaN


In [ ]:
""" df = px.data.medals_long()

fig = go.Figure(
    data=[
        go.Bar(x=medals_total.Year, y=medals_total.Medal, name="Gold"),
        go.Bar(x=medals_total.Year, y=medals_total.Medal, name="Silver"),
        go.Bar(x=medals_total.Year, y=medals_total.Medal, name="Bronze"),
    ],
    layout=dict(
        barcornerradius=15,
    ),
)

pyo.iplot(fig)"""

' df = px.data.medals_long()\n\nfig = go.Figure(\n    data=[\n        go.Bar(x=medals_total.Year, y=medals_total.Medal, name="Gold"),\n        go.Bar(x=medals_total.Year, y=medals_total.Medal, name="Silver"),\n        go.Bar(x=medals_total.Year, y=medals_total.Medal, name="Bronze"),\n    ],\n    layout=dict(\n        barcornerradius=15,\n    ),\n)\n\npyo.iplot(fig)'

In [ ]:
#merged.info()

In [ ]:
#merged

In [ ]:
#merged_ANZ = merged[merged["NOC"] == "ANZ"]

In [ ]:
#merged_ANZ

### Checking if New Zealand i specified in region for ANZ

In [ ]:
#merged_ANZ['region'].isin(["New Zealand"]).any()


In [ ]:

bronze_medal = medals_filtered.groupby(["Medal", "Year"])["NOC"].value_counts().loc[["Bronze"]]
bronze_aus = bronze_medal.xs("AUS", level="NOC")
bronze_aus

medals["Medal"] = pd.Categorical(
    medals["Medal"],
    categories=["Gold", "Silver", "Bronze"],
    ordered=True
)

medals

NameError: name 'medals_filtered' is not defined